In [8]:
%pip install pyproj
import pyproj

You should consider upgrading via the 'c:\Users\gymdo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [49]:
import folium
import pandas as pd
import numpy
from folium.plugins import MarkerCluster
#패키지 import

In [16]:
df = pd.read_csv('./data/편의시설/부산광역시_종합병원 현황_20211231.csv'
, encoding='UTF-8') #부산 경로당 현황 읽어오기 
hos_sub_df = df[df['도로명주소'].str.contains('기장군')]#관리기관명이 기장군인 것만 자름
print(hos_sub_df[['위도','경도','의료기관명']])


           위도          경도      의료기관명
27  35.321839  129.244334  동남권원자력의학원


In [50]:
df = pd.read_csv('./data/편의시설/fulldata_01_01_06_P_약국.csv'
, engine='python',encoding='cp949') #전국 약국 현황 읽어오기 
newDF=df.loc[df['소재지전체주소'].str.contains('기장군',na=False)]#기장만 끊어서
proj_UMTK = pyproj.Proj('epsg:2096') #오래된 지리원 표준
proj_WGS84 = pyproj.Proj('epsg:4326') #경위도 좌표계
newPGS = pd.DataFrame({'위도':[],'경도':[]})
for x1, y1 in zip(newDF['좌표정보(x)'],df['좌표정보(y)']):
    x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,x1,y1) #좌표 변환
    dataToInsert = pd.DataFrame({'위도':[x2],'경도':[y2]})
    newPGS = pd.concat([newPGS,dataToInsert],ignore_index=True)
newPGS = newPGS.replace([numpy.inf],numpy.nan)#inf nan으로 변환
newPGS=newPGS.dropna()#nan 제거
print(newPGS.head(10))


           위도          경도
1   37.073276  128.839780
2   37.083110  128.828849
5   37.116830  128.857980
6   37.115332  128.832001
7   37.126666  128.859012
8   37.081822  128.840460
9   37.085435  128.840548
10  37.078478  128.855626
11  37.136050  128.842022
12  37.115377  128.858302


C:\Users\gymdo\AppData\Local\Temp\ipykernel_7300\3144851901.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,x1,y1) #좌표 변환


In [17]:
kizang_pos= [35.23876594878238, 129.21575390899562] ## 기장읍 사무소
m = folium.Map(location=kizang_pos, zoom_start=17,width=750, 
               height=500)#지도 17의 줌으로 생성               
coords = hos_sub_df[['위도','경도','의료기관명']]#자료에서 위도, 경도, 경로당명 가져와서
coords2 =
hos_marker_cluster = MarkerCluster().add_to(m)#MarkerCluster을 지도에 추가하고
for lat,long,tool in zip(coords['위도'],coords['경도'],coords['의료기관명']):#lat, long, tool을 이터레이터로 만들어서
    folium.Marker([lat,long],icon=folium.Icon(color='green'),tooltip=tool).add_to(hos_marker_cluster)#초록색 마커로 마우스 갖다대면 이름 보이게 표시
m
